# Árboles de decisión y redes Bayesianas para predecir el ganador de un duelo en el juego League of Legends

In [163]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score


In [164]:
data = pd.read_csv("games.csv", sep=";")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50286 entries, 0 to 50285
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gameDuration        50286 non-null  int64
 1   firstBlood          50286 non-null  int64
 2   firstTower          50286 non-null  int64
 3   firstInhibitor      50286 non-null  int64
 4   firstBaron          50286 non-null  int64
 5   firstDragon         50286 non-null  int64
 6   firstRiftHerald     50286 non-null  int64
 7   t1_towerKills       50286 non-null  int64
 8   t1_inhibitorKills   50286 non-null  int64
 9   t1_baronKills       50286 non-null  int64
 10  t1_dragonKills      50286 non-null  int64
 11  t1_riftHeraldKills  50286 non-null  int64
 12  t2_towerKills       50286 non-null  int64
 13  t2_inhibitorKills   50286 non-null  int64
 14  t2_baronKills       50286 non-null  int64
 15  t2_dragonKills      50286 non-null  int64
 16  t2_riftHeraldKills  50286 non-null  int6

In [165]:
data.head()

,gameDuration,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,winner
0,1949,2,1,1,1,1,2,11,1,2,3,0,5,0,0,1,1,1
1,1851,1,1,1,0,1,1,10,4,0,2,1,2,0,0,0,0,1
2,1493,2,1,1,1,2,0,8,1,1,1,0,2,0,0,1,0,1
3,1758,1,1,1,1,1,0,9,2,1,2,0,0,0,0,0,0,1
4,2094,2,1,1,1,1,0,9,2,1,3,0,3,0,0,1,0,1


# Descripción estadística

In [166]:
data.describe()

,gameDuration,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,winner
count,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000
mean,1871.386032,1.487233,1.485125,1.339578,0.948693,1.477270,0.749195,5.835163,1.041781,0.381200,1.420356,0.257487,5.681999,1.008611,0.424472,1.437975,0.245854,1.493776
std,450.876052,0.499842,0.500499,0.652960,0.838988,0.530346,0.824393,3.740575,1.269064,0.588001,1.201740,0.437254,3.809336,1.261867,0.617672,1.219412,0.430597,0.499966
min,610.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1554.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1846.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,1.000000,0.000000,1.000000,0.000000,6.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,2156.750000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,9.000000,2.000000,1.000000,2.000000,1.000000,9.000000,2.000000,1.000000,2.000000,0.000000,2.000000
max,4728.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,11.000000,10.000000,5.000000,6.000000,1.000000,11.000000,10.000000,4.000000,6.000000,1.000000,2.000000


# Se separan los datos (80% de entrenamiento y 20% de prueba)

In [167]:
from sklearn.model_selection import train_test_split 
N=len(data)
cTrain=int(N*0.8) # 80% para entrenar y 20% para probar
cTest=N-cTrain
print(N,cTrain,cTest)
train_data,test_data= sklearn.model_selection.train_test_split(data, train_size=cTrain, test_size=cTest)

50286 40228 10058


In [168]:
train_data.shape

(40228, 18)

In [169]:
train_data.head()

,gameDuration,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,winner
48221,1996,2,1,1,2,2,1,9,2,0,2,1,4,0,1,2,0,1
48008,1995,2,1,1,1,2,2,11,3,1,1,0,5,0,0,1,1,1
18661,1781,1,1,1,1,1,0,11,4,1,3,0,1,0,0,0,0,1
21468,1795,1,1,1,0,1,0,8,1,0,2,0,2,0,0,1,0,1
5329,2012,1,1,1,1,1,0,9,3,1,3,0,0,0,0,0,0,1


# Se hace un Pipeline para todos los atributos, teniendo en cuenta que todos son de tipo entero


In [170]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_attribs = ['gameDuration', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills', 't1_dragonKills', 't1_riftHeraldKills', 't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills', 't2_riftHeraldKills']
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()) 
    
    ])

In [171]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
])

In [172]:
X_train = full_pipeline.fit_transform(train_data)

X_train[0,:]

array([ 0.28345244,  1.02201822, -0.96790333, -0.52341579,  1.25934199,
        0.98382999,  0.30165338,  0.84659257,  0.75604213, -0.64770065,
        0.48723988,  1.69485847, -0.44092548, -0.80010048,  0.94561527,
        0.46367099, -0.57199209])

# Se extrae las etiquetas de clase

In [173]:
y_train = train_data['winner']
y_train

48221    1
48008    1
18661    1
21468    1
5329     1
        ..
46920    2
36528    1
33019    2
15220    2
34937    1
Name: winner, Length: 40228, dtype: int64

# Se crean los áboles de decisión


# Hiperparámetros criterion=gini, splitter=best, y random_state=123.

In [174]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

modelos=np.array([None]*10)
j=0
for i in range (100, 1100, 100):
    modelos[j]= tree.DecisionTreeClassifier(criterion='gini', max_depth=i, splitter='best', random_state=123)
    modelos[j].fit(X_train, y_train)
    scores = cross_val_score(modelos[j], X_train, y_train, cv=5,scoring='accuracy')
    print(scores)
    print(scores.mean())
    j+=1

#tree.plot_tree(modelo1)
#tree.export_graphviz(decision_tree=modelo1, class_names=True, out_file='Arbol league of legends1.dot')

[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509
[0.96917723 0.97203579 0.97153865 0.97228092 0.97103791]
0.9712141018849509


In [175]:
X_test = full_pipeline.transform(test_data)
X_test

array([[-0.52592129, -0.97845614,  1.02988093, ..., -0.68449319,
        -1.180233  , -0.57199209],
       [ 1.3737658 , -0.97845614, -0.96790333, ...,  0.94561527,
        -0.358281  , -0.57199209],
       [-1.38211829, -0.97845614, -0.96790333, ..., -0.68449319,
        -1.180233  , -0.57199209],
       ...,
       [-1.42225252, -0.97845614, -0.96790333, ..., -0.68449319,
        -1.180233  , -0.57199209],
       [ 0.61567471,  1.02201822, -0.96790333, ..., -0.68449319,
        -1.180233  , -0.57199209],
       [ 1.73051455, -0.97845614,  1.02988093, ..., -0.68449319,
         1.28562298, -0.57199209]])

In [176]:
y_pred=np.array([None]*10)

for i in range (10):

    y_pred[i] = modelos[i].predict(X_test) 
    print(y_pred[i])

[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]


In [177]:
y_test = test_data["winner"]
y_test[:10]

7219     1
1676     2
46142    1
25906    1
39184    1
3121     2
19644    2
23396    1
49381    1
44467    1
Name: winner, dtype: int64

In [178]:
precision=[]
for i in range (10):
    precision.append([f"modelo {i+1}",accuracy_score(y_test, y_pred[i])])
    #print(precision[i])
tabla_precision=pd.DataFrame(precision, columns=["Modelo", "Precisión"])
tabla_precision

,Modelo,Precisión
0,modelo 1,0.967886
1,modelo 2,0.967886
2,modelo 3,0.967886
3,modelo 4,0.967886
4,modelo 5,0.967886
5,modelo 6,0.967886
6,modelo 7,0.967886
7,modelo 8,0.967886
8,modelo 9,0.967886
9,modelo 10,0.967886


# Hiperparámetros criterion=entropy, splitter=best, y random_state=123.

In [179]:
modelos_eb=np.array([None]*10)
j=0
for i in range (100, 1100, 100):
    modelos_eb[j]= tree.DecisionTreeClassifier(criterion='entropy', max_depth=i, splitter='best', random_state=123)
    modelos_eb[j].fit(X_train, y_train)
    scores = cross_val_score(modelos_eb[j], X_train, y_train, cv=5,scoring='accuracy')
    print(scores)
    print(scores.mean())
    j+=1

#tree.plot_tree(modelo11)
#tree.export_graphviz(decision_tree=modelo11, class_names=True, out_file='Arbol league of legends11.dot')

[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097
[0.97216008 0.97377579 0.97303008 0.97377253 0.97215662]
0.9729790188702097


In [180]:
y_pred_eb=np.array([None]*10)

for i in range (10):

    y_pred_eb[i] = modelos_eb[i].predict(X_test) 
    print(y_pred_eb[i])

[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]


In [198]:
precision_eb=[]
for i in range (10):
    precision_eb.append([f"modelo {i+1}",accuracy_score(y_test, y_pred_eb[i])])
    #print(precision[i])
tabla_precision_eb=pd.DataFrame(precision_eb, columns=["Modelo", "Precisión"])
tabla_precision_eb

,Modelo,Precisión
0,modelo 1,0.970968
1,modelo 2,0.970968
2,modelo 3,0.970968
3,modelo 4,0.970968
4,modelo 5,0.970968
5,modelo 6,0.970968
6,modelo 7,0.970968
7,modelo 8,0.970968
8,modelo 9,0.970968
9,modelo 10,0.970968


In [202]:
comparacion_gb_eb = tabla_precision_eb.copy()
comparacion_gb_eb.rename(columns={'Precisión':'Precisión EB'}, inplace=True)
comparacion_gb_eb['Precisión GB'] = tabla_precision['Precisión']
comparacion_gb_eb['Diferencia'] = comparacion_gb_eb['Precisión EB'] - comparacion_gb_eb['Precisión GB']
comparacion_gb_eb

,Modelo,Precisión EB,Precisión GB,Diferencia
0,modelo 1,0.970968,0.967886,0.003082
1,modelo 2,0.970968,0.967886,0.003082
2,modelo 3,0.970968,0.967886,0.003082
3,modelo 4,0.970968,0.967886,0.003082
4,modelo 5,0.970968,0.967886,0.003082
5,modelo 6,0.970968,0.967886,0.003082
6,modelo 7,0.970968,0.967886,0.003082
7,modelo 8,0.970968,0.967886,0.003082
8,modelo 9,0.970968,0.967886,0.003082
9,modelo 10,0.970968,0.967886,0.003082


## Comparación entre las precisiones usando hiperparámetros criterion=gini, splitter=best, y random_state=123 (gb) y  criterion=entropy, splitter=best, y random_state=123 (eb)

Se puede observar que la precisión usando el hiperparámetro entropy es mejor que la que se obtiene usando el hiperparámetro gini para el criterio en los modelos. Además, se evidencia que la precisión para los 10 modelos en cada caso (usando gini y entropy) es igual y por tanto sus diferencias también lo son. Por último, en la última columna se observa que es una diferencia poco significativa debido a que es del orden de las milésimas.

In [182]:
modelos_er=np.array([None]*10)
j=0
for i in range (100, 1100, 100):
    modelos_er[j]= tree.DecisionTreeClassifier(criterion='entropy', max_depth=i, splitter='random', random_state=123)
    modelos_er[j].fit(X_train, y_train)
    scores = cross_val_score(modelos_er[j], X_train, y_train, cv=5,scoring='accuracy')
    print(scores)
    print(scores.mean())
    j+=1

#tree.plot_tree(modelo21)
#tree.export_graphviz(decision_tree=modelo21, class_names=True, out_file='Arbol league of legends21.dot')

[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859
[0.97240865 0.97066866 0.97340293 0.97327533 0.9696706 ]
0.9718852335552859


In [183]:
y_pred_er=np.array([None]*10)

for i in range (10):

    y_pred_er[i] = modelos_er[i].predict(X_test) 
    print(y_pred_er[i])

[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]
[1 2 1 ... 1 1 2]


In [184]:
precision_er=[]
for i in range (10):
    precision_er.append([f"modelo {i+1}",accuracy_score(y_test, y_pred_er[i])])
    #print(precision[i])
tabla_precision_er=pd.DataFrame(precision_er, columns=["Modelo", "Precisión"])
tabla_precision_er

,Modelo,Precisión
0,modelo 1,0.968483
1,modelo 2,0.968483
2,modelo 3,0.968483
3,modelo 4,0.968483
4,modelo 5,0.968483
5,modelo 6,0.968483
6,modelo 7,0.968483
7,modelo 8,0.968483
8,modelo 9,0.968483
9,modelo 10,0.968483


In [205]:
comparacion_gb_er = tabla_precision_er.copy()
comparacion_gb_er.rename(columns={'Precisión':'Precisión ER'}, inplace=True)
comparacion_gb_er['Precisión GB'] = tabla_precision['Precisión']
comparacion_gb_er['Diferencia'] = comparacion_gb_er['Precisión ER'] - comparacion_gb_er['Precisión GB']
comparacion_gb_er

,Modelo,Precisión ER,Precisión GB,Diferencia
0,modelo 1,0.968483,0.967886,0.000597
1,modelo 2,0.968483,0.967886,0.000597
2,modelo 3,0.968483,0.967886,0.000597
3,modelo 4,0.968483,0.967886,0.000597
4,modelo 5,0.968483,0.967886,0.000597
5,modelo 6,0.968483,0.967886,0.000597
6,modelo 7,0.968483,0.967886,0.000597
7,modelo 8,0.968483,0.967886,0.000597
8,modelo 9,0.968483,0.967886,0.000597
9,modelo 10,0.968483,0.967886,0.000597


## Comparación entre las precisiones usando hiperparámetros criterion=gini, splitter=best, y random_state=123 (gb) y  criterion=entropy, splitter=random, y random_state=123 (er)

Se puede observar que la precisión usando los hiperparámetros entropy y random para el criterion y splitter en los modelos es mejor que la que se obtiene usando los hiperparámetros gini y best. Por otro lado, se evidencia que al igual que antes la precisión para los 10 modelos en cada caso (usando gini y best y entropy y random) es igual y por tanto sus diferencias también lo son. Finalmente, en la última columna se observa que es una diferencia menos significativa debido a que es del orden de las diezmilésimas, esto es, ambas precisiones son muy parecidas.

---
# Árbol con Mayor Accuracy

La mayor precisión es de 0.970968, la cual se obtuvo con los siguientes hiperparámetros:

- criterion=entropy
- splitter=best
- random_state=123
- max_depth >= 100

Donde el hiperparámetro max_depth puede ser mayor o igual a 100 debido a que a partir de ahí se obtienen las mismas precisiones.

# Hiperparámetro adicional: min_samples_leaf

## Primera Variación: min_samples_leaf=100

- criterion=entropy
- splitter=best
- random_state=123
- max_depth = 100

- **min_samples_leaf=100**

In [258]:
modelo_min_sl = tree.DecisionTreeClassifier(criterion='entropy', max_depth=100, splitter='best', random_state=123, min_samples_leaf=100)
modelo_min_sl.fit(X_train, y_train)
scores = cross_val_score(modelo_min_sl, X_train, y_train, cv=5,scoring='accuracy')
#print(scores)
#print(scores.mean())
y_pred_min_sl = modelo_min_sl.predict(X_test) 
#print(y_pred_min_sl)
accuracy_score(y_test, y_pred_min_sl)

0.9712666534102207

## Análisis del árbol de decisión primera variación

Se observa que con respecto al árbol de mayor precisión obtenida hasta el momento hubo una mejora con el hiperparámetro min_samples_leaf igual a 100, debido a que se consiguió una precisión de 0.9712666534102207 la cual es mayor a 0.970968 obtenida anteriormente (punto 8).

___
## Segunda Variación: min_samples_leaf=80

- criterion=entropy
- splitter=best
- random_state=123
- max_depth = 100
- **min_samples_leaf=80**

In [259]:
modelo_min_sl_2 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=100, splitter='best', random_state=123, min_samples_leaf=80)
modelo_min_sl_2.fit(X_train, y_train)
scores = cross_val_score(modelo_min_sl_2, X_train, y_train, cv=5,scoring='accuracy')
#print(scores)
#print(scores.mean())
y_pred_min_sl_2 = modelo_min_sl_2.predict(X_test) 
#print(y_pred_min_sl)
accuracy_score(y_test, y_pred_min_sl_2)

0.9736528136806523

## Análisis del árbol de decisión segunda variación

Al igual que antes, se observa que con respecto al árbol del punto 8 hubo una mejora con el hiperparámetro min_samples_leaf igual a 80, debido a que se consiguió una precisión de 0.9736528136806523 que es mayor a 0.970968 obtenida en ése punto.

---
# Modelo usando el clasificador Bayesiano BernoulliNB